In [1]:
import datetime as dt

import matplotlib.pyplot as plt
import mplfinance as mpf
import numpy as np
import pandas as pd
import pandas_market_calendars as mcal
import plotly.graph_objects as go
import polars as pl
from dash import Dash, dcc, html
from plotly.subplots import make_subplots

nse = mcal.get_calendar("NSE")

In [2]:
pd.set_option("display.max_rows", 25_000)
pd.set_option("display.max_columns", 500)
pl.Config.set_tbl_cols(500)
pl.Config.set_tbl_rows(10_000)

pd.options.display.float_format = "{:.4f}".format

In [3]:
import sys

sys.path.append("..")
from tooling.enums import AssetClass, Index, Spot, StrikeSpread
from tooling.fetch import fetch_option_data, fetch_spot_data
from tooling.filter import find_atm, option_tool

In [4]:
def get_expiry(f_today):
    days_to_thursday = (3 - f_today.weekday()) % 7
    nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
    f_expiry = nearest_thursday
    if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
        f_expiry = nearest_thursday - dt.timedelta(days=1)
    return f_expiry


def get_option_contract_name(symbol, strike, expiry, opt_type):
    temp = "0"
    mth = expiry.month

    if (expiry + dt.timedelta(days=7)).month != expiry.month:
        date_string = expiry.strftime("%y%b").upper()
        return f"{symbol}{date_string}{strike}{opt_type}"
    else:
        if expiry.day <= 9:
            date_string = f"{expiry.year - 2000}{mth}{temp}{expiry.day}"
        else:
            date_string = f"{expiry.year - 2000}{mth}{expiry.day}"
        return f"{symbol}{date_string}{strike}{opt_type}"

In [5]:
bnf_pandas = pd.read_csv("../data/bnf_wave.csv")

# INDEX = 'MIDCP'
INDEX = 'BNF'
# INDEX = 'NIFTY'
# INDEX = 'FNF'
# INDEX = 'STOCK'
# INDEX = 'SENSEX'

if INDEX == 'BNF':
    lev_ = 5.066
    slippage_ = 0.0001
    lot_size_ = 15
elif INDEX == 'NIFTY' or INDEX == 'FNF':
    lev_ = 6.333
    slippage_ = 0.0002
    lot_size_ = 25
elif INDEX == 'MIDCP':
    lev_ = 6.333
    slippage_ = 0.0005
    lot_size_ = 50
elif INDEX == 'SENSEX':
    lev_ = 7
    slippage_ = 0.0001
    lot_size_ = 10
elif INDEX == 'STOCK':
    lev_ = 5
    slippage_ = 0.001
    lot_size_ = 1
# bnf_pandas = pd.read_csv("../data/finnifty_1hr_tv (2).csv")
# bnf_pandas = pd.read_csv('../data/midcp_1hr_tv (4).csv')
# bnf_pandas = pd.read_csv('../data/finnifty_1hr_tv.csv')
# bnf_pandas = pd.read_csv('../data/bnf_fut_1hr_tv.csv')
# bnf_pandas = pd.read_csv('../data/gold_4hr_tv.csv')


In [6]:
# If Stocks Data ...
bnf_pandas["datetime"] = pd.to_datetime(bnf_pandas["time"])
bnf_pandas["datetime"] = bnf_pandas["datetime"].dt.tz_localize(None)
bnf_pandas = bnf_pandas[bnf_pandas["datetime"].dt.year >= 2015]
bnf_pandas.drop(columns=["time"], inplace=True)
# bnf_pandas

In [7]:
bnf = pl.DataFrame(bnf_pandas)
print(type(bnf))
# bnf
bnf = bnf.with_columns([pl.col("datetime").alias("index")]).drop("datetime")
bnf = bnf.with_columns(pl.col("index").alias("datetime"))
bnf.tail()
bnf_pandas = bnf.to_pandas()

<class 'polars.dataframe.frame.DataFrame'>


In [8]:
def resample(
    data: pl.DataFrame, timeframe, offset: dt.timedelta | None = None
) -> pl.DataFrame:
    return (
        data.set_sorted("datetime")
        .group_by_dynamic(
            index_column="datetime",
            every=timeframe,
            period=timeframe,
            label="left",
            offset=offset,
        )
        .agg(
            [
                pl.col("open").first().alias("open"),
                pl.col("high").max().alias("high"),
                pl.col("low").min().alias("low"),
                pl.col("close").last().alias("close"),
                # pl.col("volume").sum().alias("volume"),
            ]
        )
    )


In [9]:
# Parameters
ema_short_period = 9
ema_long_period = 21
rsi_period = 14
rsi_short_threshold = 50
rsi_oversold = 30
# stop_loss_pct = 0.5 / 100  # 0.5%
# take_profit_pct = 1.0 / 100  # 1.0%

In [10]:
df = bnf_pandas

In [11]:
# df["ema_short"] = df["close"].ewm(span=ema_short_period, adjust=False).mean()
# df["ema_long"] = df["close"].ewm(span=ema_long_period, adjust=False).mean()

In [12]:
# Calculate RSI
# delta = df["close"].diff()
# gain = delta.where(delta > 0, 0).rolling(window=rsi_period).mean()
# loss = -delta.where(delta < 0, 0).rolling(window=rsi_period).mean()
# rs = gain / loss
# df["rsi"] = 100 - (100 / (1 + rs))

In [33]:
def short(ema_short_period, ema_long_period, rsi_period, rsi_short_threshold, rsi_oversold):

    df["ema_short"] = df["close"].ewm(span=ema_short_period, adjust=False).mean()
    df["ema_long"] = df["close"].ewm(span=ema_long_period, adjust=False).mean()

    # Calculate RSI
    delta = df["close"].diff()
    gain = delta.where(delta > 0, 0).rolling(window=rsi_period).mean()
    loss = -delta.where(delta < 0, 0).rolling(window=rsi_period).mean()
    rs = gain / loss
    df["rsi"] = 100 - (100 / (1 + rs))
    # print(df.tail(3500).to_string())
    
    # Initialize variables
    position = None  # None, 'short'
    tradebook = []
    signal_time = None
    signal_price = None
    entry_time = None
    entry_price = None
    initial_sl = None
    
    # Iterate through rows to generate signals
    for i in range(len(df)):
        row = df.iloc[i]
    
        # Check for short signal condition
        if position is None:
            if row["ema_short"] < row["ema_long"] and row["rsi"] < rsi_short_threshold:
                if signal_price is None:  # New signal
                    signal_time = row["datetime"]
                    signal_price = row["low"]
                    # print(f'New Signal : {signal_time} , {signal_price}')
                else:
                    if row['low'] >= signal_price:
                        # Update Signal Price to Better Candle low
                        signal_time = row["datetime"]
                        signal_price = row["low"]
    
            if signal_price is not None:
                if row["low"] < signal_price and row['high'] >= signal_price:  # Signal broken
                    entry_time = row["datetime"]
                    entry_price = signal_price
                    initial_sl = max(df["high"][max(0, i - 3):i].max(), row["high"])
                    position = "short"
        
                    # Add trade to tradebook
                    tradebook.append({
                        "Signal Time": signal_time,
                        "Entry Time": entry_time,
                        "Entry Price": entry_price,
                        "Initial SL": initial_sl,
                        "Exit Time": None,
                        "Exit Price": None,
                        "Points Captured": None
                    })
                    signal_time, signal_price = None, None  # Reset signal
                else:
                    signal_price = None
                    if row["ema_short"] < row["ema_long"] and row["rsi"] < rsi_short_threshold:
                        if signal_price is None:  # New signal
                            signal_time = row["datetime"]
                            signal_price = row["low"]
                        else:
                            if row['low'] >= signal_price:
                                # Update Signal Price to Better Candle low
                                signal_time = row["datetime"]
                                signal_price = row["low"]
    
        # Exit logic for short position
        if position is not None:
            if row['close'] >= initial_sl:
                exit_time = row["datetime"]
                exit_price = row['close']
                points_captured = entry_price - exit_price
    
                # Update tradebook
                tradebook[-1]["Exit Time"] = exit_time
                tradebook[-1]["Exit Price"] = exit_price
                tradebook[-1]["Points Captured"] = points_captured
                tradebook[-1]["Remark"] = 'Initial SL Hit'
    
                # Reset position
                position = None
                entry_time, entry_price, initial_sl = None, None, None
            elif row["rsi"] > rsi_oversold:
                exit_time = row["datetime"]
                exit_price = row["close"]
                points_captured = entry_price - exit_price
    
                # Update tradebook
                tradebook[-1]["Exit Time"] = exit_time
                tradebook[-1]["Exit Price"] = exit_price
                tradebook[-1]["Points Captured"] = points_captured
                tradebook[-1]["Remark"] = 'RSI Oversold'
    
                # Reset position
                position = None
                entry_time, entry_price, initial_sl = None, None, None
    
    # Convert tradebook to a DataFrame
    tradebook_df = pd.DataFrame(tradebook)
    return tradebook_df
    # Output
    # print(tradebook_df)
    
    # Optional: Save to CSV
    # tradebook_df.to_csv("tradebook.csv", index=False)


In [34]:
ema_short_period = 5
ema_long_period = 7
rsi_period = 6
rsi_short_threshold = 60
rsi_oversold = 50
tradebook_df = short(ema_short_period, ema_long_period, rsi_period, rsi_short_threshold, rsi_oversold)
# tradebook_df

In [35]:
tradebook_df['Qty'] = 5 * 10000000 / tradebook_df['Entry Price'] #5x Lev, 1Cr portfolio
tradebook_df['PnL'] = tradebook_df['Points Captured'] * tradebook_df['Qty']
tradebook_df['ROI%'] = tradebook_df['PnL'] * 100 / 10000000

In [36]:
tradebook_df['Trade Year'] = tradebook_df['Entry Time'].dt.year

In [37]:
tradebook_df['ROI%'].sum()

441.73807864183937

In [38]:
tradebook_df[tradebook_df['Trade Year'] >= 2024]

,Signal Time,Entry Time,Entry Price,Initial SL,Exit Time,Exit Price,Points Captured,Remark,Qty,PnL,ROI%,Trade Year
1090,2024-01-01 14:45:00,2024-01-01 15:15:00,48164.1500,48450.0000,2024-01-02 13:45:00,47867.8000,296.3500,RSI Oversold,1038.1165,307645.8320,3.0765,2024
1091,2024-01-02 14:15:00,2024-01-02 14:45:00,47807.4500,47889.3000,2024-01-03 12:15:00,47631.8000,175.6500,RSI Oversold,1045.8621,183705.6777,1.8371,2024
1092,2024-01-05 12:15:00,2024-01-05 12:45:00,48074.0500,48151.4500,2024-01-05 14:45:00,48167.6000,-93.5500,Initial SL Hit,1040.0622,-97297.8145,-0.9730,2024
1093,2024-01-05 15:15:00,2024-01-08 09:15:00,48043.0500,48264.1000,2024-01-08 10:15:00,48019.1500,23.9000,RSI Oversold,1040.7333,24873.5249,0.2487,2024
1094,2024-01-08 11:15:00,2024-01-08 11:45:00,47853.3500,48082.1000,2024-01-09 10:15:00,47882.7500,-29.4000,RSI Oversold,1044.8589,-30718.8525,-0.3072,2024
1095,2024-01-09 13:45:00,2024-01-09 14:15:00,47655.8000,47874.2500,2024-01-10 10:45:00,47275.4000,380.4000,RSI Oversold,1049.1902,399111.9654,3.9911,2024
1096,2024-01-10 11:15:00,2024-01-10 11:45:00,47256.6000,47386.4500,2024-01-10 11:45:00,47263.6000,-7.0000,RSI Oversold,1058.0533,-7406.3729,-0.0741,2024
1097,2024-01-10 12:15:00,2024-01-10 12:45:00,47227.1500,47355.6000,2024-01-10 13:15:00,47267.1000,-39.9500,RSI Oversold,1058.7130,-42295.5863,-0.4230,2024
1098,2024-01-11 13:45:00,2024-01-11 14:15:00,47260.8500,47595.1500,2024-01-12 09:45:00,47629.6000,-368.7500,Initial SL Hit,1057.9581,-390122.0566,-3.9012,2024
1099,2024-01-16 12:45:00,2024-01-16 13:15:00,48091.5000,48305.4000,2024-01-18 11:45:00,46170.7500,1920.7500,RSI Oversold,1039.6848,1996974.5173,19.9697,2024


In [39]:
def generate_stats(tb_expiry, variation):
    stats_df8 = pd.DataFrame(
        index=range(2018, 2025),
        columns=[
            "Total ROI",
            "Total Trades",
            "Win Rate",
            "Avg Profit% per Trade",
            "Avg Loss% per Trade",
            "Max Drawdown",
            "ROI/DD Ratio",
            "Variation",
        ],
    )
    combined_df_sorted = tb_expiry
    # combined_df_sorted = tb_expiry_ce
    # combined_df_sorted = tb_expiry_pe
    
    # Iterate over each year
    for year in range(2018, 2025):
        # Filter trades for the current year
        year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]
    
        # Calculate total ROI
        total_roi = year_trades["ROI%"].sum()
    
        # Calculate total number of trades
        total_trades = len(year_trades)
    
        # Calculate win rate
        win_rate = (year_trades["ROI%"] > 0).mean() * 100
    
        # Calculate average profit per trade
        avg_profit = year_trades[year_trades["ROI%"] > 0]["ROI%"].mean()
    
        # Calculate average loss per trade
        avg_loss = year_trades[year_trades["ROI%"] < 0]["ROI%"].mean()
    
        # Calculate maximum drawdown
        max_drawdown = (
            year_trades["ROI%"].cumsum() - year_trades["ROI%"].cumsum().cummax()
        ).min()
    
        # Calculate ROI/DD ratio
        roi_dd_ratio = total_roi / abs(max_drawdown)

        variation = variation
    
        # Store the statistics in the DataFrame
        stats_df8.loc[year] = [
            total_roi,
            total_trades,
            win_rate,
            avg_profit,
            avg_loss,
            max_drawdown,
            roi_dd_ratio,
            variation,
        ]
    
    # Calculate overall statistics
    overall_total_roi = stats_df8["Total ROI"].sum()
    overall_total_trades = stats_df8["Total Trades"].sum()
    overall_win_rate = (combined_df_sorted["ROI%"] > 0).mean() * 100
    overall_avg_profit = combined_df_sorted[combined_df_sorted["ROI%"] > 0]["ROI%"].mean()
    overall_avg_loss = combined_df_sorted[combined_df_sorted["ROI%"] < 0]["ROI%"].mean()
    overall_max_drawdown = (
        combined_df_sorted["ROI%"].cumsum() - combined_df_sorted["ROI%"].cumsum().cummax()
    ).min()
    overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)
    overall_variation = variation
    
    # Store the overall statistics in the DataFrame
    stats_df8.loc["Overall"] = [
        overall_total_roi,
        overall_total_trades,
        overall_win_rate,
        overall_avg_profit,
        overall_avg_loss,
        overall_max_drawdown,
        overall_roi_dd_ratio,
        overall_variation,
    ]
    return {overall_roi_dd_ratio : stats_df8}

In [40]:
variation = f'...{rsi_oversold}'
stats = generate_stats(tradebook_df, variation)

In [41]:
stats_dictionary = {}
for overall_roi_dd_ratio, stats_df in stats.items():
    if overall_roi_dd_ratio is not None and overall_roi_dd_ratio > -10:
        print(stats_df.to_string())
        stats_dictionary[overall_roi_dd_ratio] = stats_df

        Total ROI Total Trades Win Rate Avg Profit% per Trade Avg Loss% per Trade Max Drawdown ROI/DD Ratio Variation
2018      52.3380          189  38.6243                2.9957             -1.4341     -22.0702       2.3714     ...50
2019      45.6430          173  36.9942                3.3902             -1.5718     -37.5853       1.2144     ...50
2020     236.9007          165  44.8485                7.6364             -3.6065     -43.0830       5.4987     ...50
2021      81.4157          193  37.8238                4.7666             -2.2212     -31.3163       2.5998     ...50
2022     -10.3375          195  36.4103                3.6273             -2.1603     -62.2904      -0.1660     ...50
2023      26.7128          175  36.5714                2.5879             -1.2515     -31.4370       0.8497     ...50
2024       9.0655          175  33.1429                3.1451             -1.4816     -45.9112       0.1975     ...50
Overall  441.7381         1265  37.7075                4

In [ ]:
stats_dictionary = {}

for ema_short_period in range(5, 16, 2):
    for ema_long_period in range(5, 24, 2):
        for rsi_period in range(2, 15, 2):
            for rsi_short_threshold in range(20, 81, 10):
                for rsi_oversold in range(20, 61, 10):
                    if (ema_short_period < ema_long_period) and (rsi_oversold <= rsi_short_threshold):
                        tradebook_df = short(ema_short_period, ema_long_period, rsi_period, rsi_short_threshold, rsi_oversold)
                        tradebook_df['Qty'] = 5 * 10000000 / tradebook_df['Entry Price'] #5x Lev, 1Cr portfolio
                        tradebook_df['PnL'] = tradebook_df['Points Captured'] * tradebook_df['Qty']
                        tradebook_df['ROI%'] = tradebook_df['PnL'] * 100 / 10000000
                        tradebook_df['Trade Year'] = tradebook_df['Entry Time'].dt.year
                        variation = f'{ema_short_period}, {ema_long_period}, {rsi_period}, {rsi_short_threshold}, {rsi_oversold} -> {round(tradebook_df["ROI%"].sum())}%'
                        print(variation)
                        new_tb = tradebook_df
                        if len(new_tb) > 1:
                            new_tb["DD%"] = new_tb["ROI%"].cumsum() - new_tb["ROI%"].cumsum().cummax()
                            stats = generate_stats(new_tb, variation)
                            for overall_roi_dd_ratio, stats_df in stats.items():
                                if overall_roi_dd_ratio is not None and overall_roi_dd_ratio > 9:
                                    print(stats_df.to_string())
                                    stats_dictionary[overall_roi_dd_ratio] = stats_df

5, 7, 2, 20, 20 -> 243%
5, 7, 2, 30, 20 -> 274%
5, 7, 2, 30, 30 -> 330%
5, 7, 2, 40, 20 -> 352%
5, 7, 2, 40, 30 -> 399%
5, 7, 2, 40, 40 -> 412%
5, 7, 2, 50, 20 -> 425%
5, 7, 2, 50, 30 -> 444%
5, 7, 2, 50, 40 -> 457%
5, 7, 2, 50, 50 -> 465%
5, 7, 2, 60, 20 -> 429%
5, 7, 2, 60, 30 -> 460%
5, 7, 2, 60, 40 -> 471%
5, 7, 2, 60, 50 -> 491%
5, 7, 2, 60, 60 -> 528%
5, 7, 2, 70, 20 -> 435%
5, 7, 2, 70, 30 -> 472%
5, 7, 2, 70, 40 -> 476%
5, 7, 2, 70, 50 -> 497%
5, 7, 2, 70, 60 -> 538%
        Total ROI Total Trades Win Rate Avg Profit% per Trade Avg Loss% per Trade Max Drawdown ROI/DD Ratio                Variation
2018      71.5597          248  36.6935                2.5862             -1.0432     -18.8521       3.7958  5, 7, 2, 70, 60 -> 538%
2019      24.7189          245  32.6531                2.7694             -1.1929     -25.5557       0.9673  5, 7, 2, 70, 60 -> 538%
2020     267.3666          241  40.6639                6.4717             -2.5655     -59.1099       4.5232  5, 7, 2, 70,

In [184]:
sorted_stats = {k: v for k, v in sorted(stats_dictionary.items(), key=lambda item: item[0], reverse=True)}
sorted_stats

{11.02707241854197:         Total ROI Total Trades Win Rate Avg Profit% per Trade  \
 2015     108.3960           58  39.6552                7.8867   
 2016      59.0682           63  38.0952                6.1991   
 2017      -7.6265           42  30.9524                2.8584   
 2018      46.0476           51  37.2549                5.4079   
 2019      42.9508           62  51.6129                3.3882   
 2020     274.0887           51  43.1373               17.2595   
 2021       5.3462           67  37.3134                4.2052   
 2022      28.9484           48  29.1667                9.8799   
 2023      -0.2518           74  33.7838                3.4593   
 2024     -23.1221           65  27.6923                3.7562   
 Overall  533.8454          581  37.0052                6.3058   
 
         Avg Loss% per Trade Max Drawdown ROI/DD Ratio  \
 2015                -2.0857     -17.2475       6.2847   
 2016                -2.3003     -36.3339       1.6257   
 2017        